In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import CommentPull.CommentPuller
from sklearn.preprocessing import RobustScaler

In [2]:
#Make graphs pretty and bigger
plt.rcParams['figure.figsize'] = (10, 10)
sns.set()

In [3]:
#Load data using custom class
comment_puller = CommentPull.CommentPuller.CommentPuller(load=True)

In [4]:
#Split title data
X_title = comment_puller.title_df.drop(['label_id', 'post_id'], axis=1)
y_title = comment_puller.title_df['label_id']
X_title_train, X_title_test, y_title_train, y_title_test = train_test_split(X_title, y_title,
                                                                           test_size=0.15, random_state=10)
#Scale training data
title_scaler = RobustScaler()
X_title_train = title_scaler.fit_transform(X_title_train)

#Scale test data in same manner
X_title_test = title_scaler.transform(X_title_test)

In [5]:
#Split text data 
X_text = comment_puller.text_df.drop(['label_id', 'post_id'], axis=1)
y_text = comment_puller.text_df['label_id']
X_text_train, X_text_test, y_text_train, y_text_test = train_test_split(X_text, y_text,
                                                                           test_size=0.15, random_state=10)
#Scale training data
text_scaler = RobustScaler()
X_text_train = text_scaler.fit_transform(X_text_train)
#Scale test data in same manner as training
X_text_test = text_scaler.transform(X_text_test)

In [19]:
#Let's begin with a Logisitic Regression Model
from sklearn.linear_model import LogisticRegression
#Intialize model
log_title = LogisticRegression(solver='newton-cg')
#Create a parameter grid over which we can gridsearch
param_grid ={'C':np.logspace(-4, 4, 10) , 'penalty':['l1', 'l2']}
#Intialize a gridsearch object
log_title_gs = GridSearchCV(log_title, param_grid, cv=5, scoring='roc_auc')
#Fit the gridsearch object
log_title_gs.fit(X_title_train, y_title_train)
#Print out best hyperparameters
print('Best parameter for n_neighbors is {}'.format(log_title_gs.best_params_))
print('Scoring {}'.format(log_title_gs.best_score_))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
50 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
Val

Best parameter for n_neighbors is {'C': 0.3593813663804626, 'penalty': 'l2'}
Scoring 0.6181010502867073


In [17]:
#Intialize model
log_text = LogisticRegression(solver='newton-cg')
#Intialize a gridsearch object
log_text_gs = GridSearchCV(log_text, param_grid, cv=5, scoring='roc_auc')
#Fit the gridsearch object
log_text_gs.fit(X_text_train, y_text_train)
#Print out best hyperparameters
print('Best parameter for n_neighbors is {}'.format(log_text_gs.best_params_))
print('Scoring {}'.format(log_text_gs.best_score_))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
50 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
Val

Best parameter for n_neighbors is {'C': 0.005994842503189409, 'penalty': 'l2'}
Scoring 0.6237225483991814


In [10]:
embeddings_index = {}
f = open('glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [49]:
#Get vocab of title_df, exclude first two elements as they are not training features
vocab = comment_puller.title_df.columns.tolist()[2:]
embedding_matrix = np.zeros((len(vocab) + 1, 100))
for i, word in enumerate(vocab):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [54]:
#Let's try a deep learning model now
import tensorflow as tf
import keras
from keras.layers import Dense, Dropout, Embedding, LSTM
from keras.models import Sequential#Try pytorch
#Intialize a model
title_deep_model = Sequential()#Try CNN, try sklearn NN mlpclassifier
#Add layers
title_deep_model.add(Embedding(len(vocab) + 1, 100, weights=[embedding_matrix], 
                               input_length=X_title_train.shape[1] , trainable=False))
title_deep_model.add(Dense(50, activation='relu', input_shape=(X_title_train.shape[1],), 
                          kernel_regularizer=keras.regularizers.l1_l2(l1=0.0001, l2=0.0001)))
title_deep_model.add(Dropout(0.3))
title_deep_model.add(Dense(25, activation='relu', 
                    kernel_regularizer=keras.regularizers.l1_l2(l1=0.001, l2=0.001)))
title_deep_model.add(Dropout(0.4))
title_deep_model.add(Dense(5, activation='relu'))
title_deep_model.add(Dropout(0.5))
title_deep_model.add(Dense(1, activation='sigmoid'))
#Compile model
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
title_deep_model.compile(optimizer=opt, loss='binary_crossentropy', 
                         metrics=['accuracy', 'AUC'])
#Fit model
title_deep_model.fit(X_title_train, y_title_train, epochs=20, verbose=2, 
                     validation_data=(X_title_test, y_title_test))

Epoch 1/20


ValueError: in user code:

    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 817, in train_step
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 460, in update_state
        metric_obj.update_state(y_t, y_p, sample_weight=mask)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\utils\metrics_utils.py", line 73, in decorated
        update_op = update_state_fn

In [97]:
#Now let's try with the text data
#Intialize a model
text_deep_model = Sequential()
#Add layers
text_deep_model.add(Dense(75, activation='relu', input_shape=(X_text_train.shape[1],)))
text_deep_model.add(Dropout(0.3))
text_deep_model.add(Dense(30, activation='relu'))#Add dropout layer
text_deep_model.add(Dropout(0.3))
text_deep_model.add(Dense(5, activation='relu'))
text_deep_model.add(Dropout(0.3))
text_deep_model.add(Dense(1, activation='sigmoid'))
#Compile model
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
text_deep_model.compile(optimizer=opt, loss='binary_crossentropy', 
                         metrics=['accuracy', 'AUC'])
#Fit model
text_deep_model.fit(X_text_train, y_text_train, epochs=3, verbose=2, 
                     validation_data=(X_text_test, y_text_test))

Epoch 1/3
95/95 - 1s - loss: 0.4532 - accuracy: 0.8696 - auc: 0.5506 - val_loss: 0.3384 - val_accuracy: 0.8972 - val_auc: 0.6027 - 1s/epoch - 15ms/step
Epoch 2/3
95/95 - 1s - loss: 0.3351 - accuracy: 0.8818 - auc: 0.7270 - val_loss: 0.3410 - val_accuracy: 0.8972 - val_auc: 0.5996 - 547ms/epoch - 6ms/step
Epoch 3/3
95/95 - 1s - loss: 0.2113 - accuracy: 0.9241 - auc: 0.9190 - val_loss: 0.5109 - val_accuracy: 0.8935 - val_auc: 0.5914 - 523ms/epoch - 6ms/step


These models don't seem to be improving on much. This may be because there isn't enough data, the data doesn't generalize well or some other reason. The BERT training model is a huge model that has already been trained on millions of data examples. Let's see if we can tweak this model and use it ourselves.

In [32]:
#For this bert model we need to pass the actual text columns
#First let's get title text and labels
X_title_bert = comment_puller.post_df['title'].apply(comment_puller._clean_string)
y_title_bert = comment_puller.title_df['label_id']
#We then split these
X_title_train_bert, X_title_test_bert, y_title_train_bert, y_title_test_bert = train_test_split(
X_title_bert, y_title_bert, test_size=0.15, random_state=10)
#We now do the same for the text body of posts
X_text_bert = comment_puller.post_df['post_text'].apply(comment_puller._clean_string)
y_text_bert = comment_puller.text_df['label_id']
#Split them here
X_text_train_bert, X_text_test_bert, y_text_train_bert, y_text_test_bert = train_test_split(
X_text_bert, y_text_bert, test_size=0.15, random_state=10)

In [100]:
X_text_bert.apply(lambda x: x.split()) #Throw into BertTokenClassifier

0       [brother, inlaw, sammy, lost, home, shortly, d...
1       [ill, try, keep, short, impala, doorhttpsiimgu...
2       [high, schooler, weekend, job, coffee, shop, c...
3       [week, ago, f, family, ordered, chinese, food,...
4       [dad, old, fart, love, daughter, piece, ’, str...
                              ...                        
3559    [screwed, shot, mouth, facebook, said, somethi...
3560    [genderfluid, moved, away, transferred, univer...
3561    [bf, work, shipping, facility, us, car, get, w...
3562    [asshole, wanting, go, funeral, birth, father,...
3563    [self, explanatory, started, anti, depressant,...
Name: post_text, Length: 3564, dtype: object

In [33]:
#Import sklearn-like wrapper for BERT model
from bert_sklearn import BertClassifier 
#Intialize model
bert_title_model = BertClassifier()
#Adjust batch size so it doesn't eat all GPU memory
bert_title_model.train_batch_size=16
#Tune model
bert_title_model.fit(X_title_train_bert, y_title_train_bert)
#Score data
bert_title_model.score(X_title_test_bert, y_title_test_bert)

Building sklearn text classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 2727, validation data size: 302


Validating: 100%|██████████████████████████████████████████████████████████████████████| 38/38 [00:09<00:00,  3.86it/s]

Epoch 1, Train loss: 0.3729, Val loss: 0.4434, Val accy: 84.77%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 38/38 [00:09<00:00,  3.84it/s]

Epoch 2, Train loss: 0.3591, Val loss: 0.4295, Val accy: 84.77%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 38/38 [00:09<00:00,  3.84it/s]

Epoch 3, Train loss: 0.3360, Val loss: 0.4360, Val accy: 84.77%



Testing: 100%|█████████████████████████████████████████████████████████████████████████| 67/67 [00:10<00:00,  6.13it/s]


Loss: 0.3323, Accuracy: 89.72%


89.7196261682243

In [37]:
from sklearn.metrics import roc_auc_score
title_preds = bert_title_model.predict(X_title_test)
roc_auc_score(y_title_test_bert, title_preds)

Predicting: 100%|██████████████████████████████████████████████████████████████████████| 67/67 [00:11<00:00,  6.06it/s]


0.5

In [38]:
#Intialize model
bert_text_model = BertClassifier()
#Try throwing in just after cleaning
#Then try only from df vocab
#Adjust batch size so it doesn't eat all GPU memory
bert_text_model.train_batch_size=16
#Tune model
bert_text_model.fit(X_text_train_bert, y_text_train_bert)
#Score data
bert_text_model.score(X_text_test_bert, y_text_test_bert)

Building sklearn text classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 2727, validation data size: 302


Validating: 100%|██████████████████████████████████████████████████████████████████████| 38/38 [00:09<00:00,  3.81it/s]

Epoch 1, Train loss: 0.3721, Val loss: 0.4449, Val accy: 84.77%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 38/38 [00:09<00:00,  3.81it/s]

Epoch 2, Train loss: 0.3603, Val loss: 0.4318, Val accy: 84.77%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 38/38 [00:10<00:00,  3.80it/s]

Epoch 3, Train loss: 0.3595, Val loss: 0.4319, Val accy: 84.77%



Testing: 100%|█████████████████████████████████████████████████████████████████████████| 67/67 [00:10<00:00,  6.10it/s]


Loss: 0.3318, Accuracy: 89.72%


89.7196261682243

In [103]:
text_preds = bert_text_model.predict_proba(X_text_test_bert)#Try another transfer learning

Predicting: 100%|██████████████████████████████████████████████████████████████████████| 67/67 [00:12<00:00,  5.57it/s]


In [104]:
roc_auc_score(y_text_test_bert, text_preds[:, 1])

0.5156439393939394

In [13]:
BertClassifier().epochs

Building sklearn text classifier...


3

In [94]:
#Open text file to write to 
f = open('training.txt', 'w', encoding='utf-8')

In [95]:
#For our chat bot, we need to create a text file of post titles, text bodies and comments.
#Create an empty string to write to
text = ""
#Make copies of df's to make code easier to read
post_df = comment_puller.post_df.copy()
comment_df = comment_puller.comment_df.copy()
#Get post for specific post_id
for post_id in post_df['post_id']:
    #Retrieve the post and write the title text and body text
    post = post_df[post_df['post_id']==post_id]
    text += post['title'].values[0] + '\n'
    text += post['post_text'].values[0] + '\n'
    #Get the top ten comments of post saved in comment_df
    comments = comment_df[comment_df['post_id'].isin([post_id])]
    #Write them to the text string
    for comment in comments['comment']:
        text += comment + '/n'
    #Write the titles, text bodies and comments to our file
    f.write(text)
    #Reset text
    text = ""

In [96]:
f.close()